# Simple Dashboard demo

In [1]:
import dash
import pandas as pd
import jupyterlab_dash
import plotly
import plotly.graph_objs as go
import chart_studio.plotly as py

import dash_core_components as dcc
import dash_html_components as html
import numpy as np

In [2]:
# Local test - set up connection
%run ./db_common_local.py

conn = default_conn

In [ ]:
# Sagemaker notebook env
%run ./db_common.py

conn = default_conn

## TPCDS Query16
Report number of orders, total shipping costs and profits from web sales of particular counties and states for a given 60 day period for non-returned sales filled from an alternate warehouse.

In [3]:
query_stmt = '''
select  ca_state,
   count(distinct ws_order_number) as "order_count"
  ,avg(ws_ext_ship_cost) as "avg_shipping_cost"
  ,avg(ws_net_profit) as "avg_net_profit"
from
   web_sales cs1
  ,date_dim
  ,customer_address
where
    d_year = 2000
and cs1.ws_ship_date_sk = d_date_sk
and cs1.ws_ship_addr_sk = ca_address_sk
and cs1.ws_order_number not in (select wr_order_number
           from web_returns cr1)
group by 1
order by 2 desc;
'''

# Initialize
dataset = 'tpcds_100gb'
plotly.offline.init_notebook_mode(connected=True)

search_path = 'set search_path to %s;' % dataset
exec_dml(conn, search_path)

Escaped time: 0 seconds


0.42125415802001953

In [4]:
# Valid year range 1998-2002
query_stmt = query_stmt.replace('\n', ' ').replace('\r', '')
df = exec_query(conn, query_stmt)

# convert everything to string for display purpose
for col in df.columns: 
    df[col] = df[col].astype(str)

df['text'] = 'order ' + df['order_count'] + '<br>' + \
            'average shipping cost ' + df['avg_shipping_cost'] + '<br>' + \
            'average net profit ' + df['avg_net_profit']

fig = go.Figure(data=go.Choropleth(
    locations=df['ca_state'], # Spatial coordinates
    z = df['order_count'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    text = df['text'],
    colorscale = 'Reds',
    colorbar_title = "Total orders",
))

fig.update_layout(
    title_text = 'US Shipping cost and profit by State',
    geo_scope='usa', # limite map scope to USA
)

fig.show()

Escaped time: 0 seconds, 	Returned rows: 52
